Recreate a model based on the third place solution of the [Tick Tick Bloom: Harmful Algal Bloom Detection Challenge](https://github.com/drivendataorg/tick-tick-bloom/tree/main)

In [1]:
%load_ext lab_black
%load_ext autoreload
%autoreload 2

In [2]:
import yaml

from cloudpathlib import AnyPath

from cyano.config import FeaturesConfig, ModelTrainingConfig
from cyano.experiment import ExperimentConfig
from cyano.settings import REPO_ROOT

/opt/anaconda3/envs/cyanobacteria-prediction/lib/python3.10/site-packages/pydantic/_internal/_fields.py:126: UserWarning: Field "model_training_config" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [3]:
DATA_DIR = REPO_ROOT.parent / "data/experiments"
S3_DATA_DIR = AnyPath("s3://drivendata-competition-nasa-cyanobacteria") / "experiments"

SPLITS_DIR = S3_DATA_DIR / "splits"
LOCAL_CACHE_DIR = DATA_DIR / "cache"
EXPERIMENT_SAVE_DIR = DATA_DIR / "rerun_third"
EXPERIMENT_SAVE_DIR.mkdir(exist_ok=True, parents=True)

## Settings

Write config to match third place code

In [4]:
use_sentinel_bands = [
    "AOT",
    "B01",
    "B02",
    "B03",
    "B04",
    "B05",
    "B06",
    "B07",
    "B08",
    "B09",
    "B11",
    "B12",
    "B8A",
    "SCL",
    "WVP",
]
sat_image_fts = [
    f"{band}_{stat}"
    for band in use_sentinel_bands
    for stat in ["mean", "min", "max", "range"]
]
sat_image_fts += ["NDVI_B04", "NDVI_B05", "NDVI_B06", "NDVI_B07"]
sat_image_fts[:6]

['AOT_mean', 'AOT_min', 'AOT_max', 'AOT_range', 'B01_mean', 'B01_min']

In [5]:
len(sat_image_fts)

64

In [6]:
features_config = FeaturesConfig(
    image_feature_meter_window=200,
    n_sentinel_items=15,
    pc_meters_search_window=5000,
    pc_days_search_window=15,
    use_sentinel_bands=use_sentinel_bands,
    satellite_image_features=sat_image_fts,
    satellite_meta_features=["month", "days_before_sample"],
    metadata_features=["rounded_longitude"],
    num_threads=8,
)

In [7]:
model_config = ModelTrainingConfig(
    num_boost_round=100000,
    params={
        "application": "regression",
        "metric": "rmse",
        "max_depth": -1,
        "num_leaves": 31,
        "learning_rate": 0.1,
    },
)

In [8]:
experiment_config = ExperimentConfig(
    features_config=features_config,
    train_csv=str(SPLITS_DIR / "competition/train.csv"),
    predict_csv=str(SPLITS_DIR / "competition/test.csv"),
    cache_dir=LOCAL_CACHE_DIR,
    save_dir=EXPERIMENT_SAVE_DIR,
)

with (EXPERIMENT_SAVE_DIR / "experiment_config.yaml").open("w") as fp:
    yaml.dump(experiment_config.model_dump(), fp)

## Run experiment

In [ ]:
experiment_config.run_experiment()

2023-08-11 22:38:28.014 | INFO     | cyano.pipeline:_prep_train_data:50 - Loaded 17,060 samples for training
2023-08-11 22:38:28.017 | INFO     | cyano.data.satellite_data:generate_candidate_metadata:194 - Generating metadata for all satellite item candidates
2023-08-11 22:39:27.262 | INFO     | cyano.data.satellite_data:generate_candidate_metadata:204 - Loaded 56,173 rows of Sentinel candidate metadata from s3://drivendata-competition-nasa-cyanobacteria/data/interim/full_pc_search
2023-08-11 22:39:44.624 | INFO     | cyano.data.satellite_data:identify_satellite_data:299 - Selecting which items to use for feature generation
100%|████████████████████████████████████████████████████████████| 17060/17060 [01:36<00:00, 177.54it/s]
2023-08-11 22:41:23.154 | INFO     | cyano.data.satellite_data:identify_satellite_data:318 - Identified satellite imagery for 11,544 samples
2023-08-11 22:41:28.686 | INFO     | cyano.pipeline:_prepare_features:60 - 45,210 rows of satellite metadata saved to /Use

  0%|          | 0/45210 [00:00<?, ?it/s]

2023-08-11 22:41:42.088 | WARNING  | cyano.data.satellite_data:download_row:392 - Other error encountered
2023-08-11 22:41:42.102 | WARNING  | cyano.data.satellite_data:download_row:392 - Other error encountered
2023-08-11 22:41:42.109 | WARNING  | cyano.data.satellite_data:download_row:392 - Other error encountered
2023-08-11 22:46:58.016 | WARNING  | cyano.data.satellite_data:download_row:392 - Other error encountered
2023-08-11 22:46:59.199 | WARNING  | cyano.data.satellite_data:download_row:392 - Other error encountered
2023-08-11 22:47:00.146 | WARNING  | cyano.data.satellite_data:download_row:392 - Other error encountered
2023-08-11 22:57:42.784 | WARNING  | cyano.data.satellite_data:download_row:392 - Other error encountered
2023-08-11 22:57:45.626 | WARNING  | cyano.data.satellite_data:download_row:392 - Other error encountered
2023-08-11 22:57:47.887 | WARNING  | cyano.data.satellite_data:download_row:392 - Other error encountered
2023-08-11 23:35:46.904 | WARNING  | cyano.dat